## VIJAY KRISHNA RV
## ME21B1039

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install --quiet \
  transformers==4.38.2 \
  datasets==2.19.1 \
  evaluate==0.4.1 \
  accelerate==0.27.2 \
  peft==0.8.2 \
  rouge-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 88.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 86.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import evaluate


2025-04-11 13:21:25.249949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744377685.443390      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744377685.503824      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Load ILSUM English subset
dataset = load_dataset("ILSUM/ILSUM-1.0", split="train", name="English")

MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

def filter_example(example):
    return len(example['Article'].split()) <= MAX_INPUT_LENGTH and len(example['Summary'].split()) <= MAX_TARGET_LENGTH

dataset = dataset.filter(filter_example)

# Train/val/test split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_valid = dataset["train"].train_test_split(test_size=0.1, seed=42)

train_dataset = train_valid["train"]
val_dataset = train_valid["test"]
test_dataset = dataset["test"]


Generating train split:   0%|          | 0/12565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4487 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12565 [00:00<?, ? examples/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

def preprocess_function(example):
    model_inputs = tokenizer(
        example["Article"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["Summary"],
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/5673 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/631 [00:00<?, ? examples/s]

Map:   0%|          | 0/1576 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [9]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,           # For testing
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,             # ⬅️ Log every 10 steps
    report_to="none"              # ⬅️ No external logging (just print)
)


In [11]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge1 = rouge2 = rougeL = 0.0
    for pred, label in zip(decoded_preds, decoded_labels):
        scores = scorer.score(label, pred)
        rouge1 += scores["rouge1"].fmeasure
        rouge2 += scores["rouge2"].fmeasure
        rougeL += scores["rougeL"].fmeasure

    total = len(decoded_preds)
    return {
        "rouge1": round(rouge1 / total * 100, 2),
        "rouge2": round(rouge2 / total * 100, 2),
        "rougeL": round(rougeL / total * 100, 2),
    }


In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [13]:
trainer.train()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,0.103300,0.103154,33.690000,23.670000,30.950000
2,0.108900,0.102206,36.590000,27.190000,33.940000
3,0.066800,0.101788,36.520000,27.410000,34.000000
4,0.089800,0.102981,36.590000,27.660000,34.150000
5,0.065500,0.103503,36.280000,27.310000,33.880000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agno

TrainOutput(global_step=7095, training_loss=0.15773942608115873, metrics={'train_runtime': 2562.8579, 'train_samples_per_second': 11.068, 'train_steps_per_second': 2.768, 'total_flos': 8647587318988800.0, 'train_loss': 0.15773942608115873, 'epoch': 5.0})

In [14]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

sample_article = """
India’s Chandrayaan-3 mission successfully landed on the Moon, marking a historic moment for the country’s space program.
The lander touched down near the lunar south pole, making India the first nation to achieve this feat.
"""

summary = summarizer(sample_article, max_length=128, min_length=30, do_sample=False)
print("📝 Generated Summary:\n", summary[0]["summary_text"])


Your max_length is set to 128, but your input_length is only 55. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)


📝 Generated Summary:
 The lander touched down near the lunar south pole, making India the first nation to achieve this feat. The Indian Space Agency said that the Chandrayaan-3 mission successfully landed on the Moon.


In [15]:
long_article = """
India’s Chandrayaan-3 mission successfully landed on the Moon, marking a historic milestone for the country’s space program. 
The lander, named Vikram, touched down near the unexplored lunar south pole — a region where no spacecraft has ever landed before. 
This makes India the first country to reach this part of the Moon. The Indian Space Research Organisation (ISRO) celebrated this achievement, 
highlighting that it demonstrates India’s growing capabilities in space exploration. The rover onboard, Pragyan, will now begin its mission 
to analyze the lunar surface for signs of water ice and important minerals. This success comes after the Chandrayaan-2 mission failed to land safely in 2019. 
Prime Minister Narendra Modi praised the mission, stating that “India is now on the Moon” and dedicating the moment to scientists and citizens alike.
"""

summary = summarizer(long_article, max_length=100, min_length=40, do_sample=False)
print("📝 Generated Summary:\n", summary[0]["summary_text"])


📝 Generated Summary:
 This makes India the first country to reach this part of the Moon. The Indian Space Research Organisation (ISRO) celebrated this achievement,  highlighting that it demonstrates India’s growing capabilities in space exploration.


In [17]:
article_1 = """
India’s Chandrayaan-3 mission successfully landed on the Moon, marking a historic milestone for the country’s space program. 
The lander, named Vikram, touched down near the unexplored lunar south pole — a region where no spacecraft has ever landed before. 
This makes India the first country to reach this part of the Moon. The Indian Space Research Organisation (ISRO) celebrated this achievement, 
highlighting that it demonstrates India’s growing capabilities in space exploration. The rover onboard, Pragyan, will now begin its mission 
to analyze the lunar surface for signs of water ice and important minerals. This success comes after the Chandrayaan-2 mission failed to land safely in 2019. 
Prime Minister Narendra Modi praised the mission, stating that “India is now on the Moon” and dedicating the moment to scientists and citizens alike.
"""
article_2 = """
In a landmark deal, over 190 countries have agreed on a new global framework to address climate change at the United Nations Climate Conference. 
The agreement sets ambitious targets for reducing carbon emissions and aims to limit global warming to 1.5 degrees Celsius above pre-industrial levels. 
Nations have pledged to move away from fossil fuels, invest in renewable energy, and protect biodiversity. 
The deal also includes a $100 billion annual fund to help developing countries adapt to climate impacts. 
While activists say more aggressive action is needed, world leaders have called the agreement a significant step toward a sustainable future.
"""
article_3 = """
Scientists have announced a major breakthrough in mRNA vaccine technology. A new vaccine has shown remarkable success in protecting against a wide range of flu strains in early human trials. 
Unlike traditional vaccines, the mRNA platform allows for faster development and easier updates to match mutating viruses. 
This advancement could revolutionize how we respond to seasonal flu and future pandemics. 
Experts believe the new technology will also enhance vaccine equity by enabling lower-cost production and simpler storage requirements. 
The study, published in the New England Journal of Medicine, is being hailed as a milestone in preventative medicine.
"""


In [18]:
articles = [article_1, article_2, article_3]

for i, article in enumerate(articles, 1):
    summary = summarizer(article, max_length=100, min_length=40, do_sample=False)
    print(f"📰 Article {i} Summary:\n{summary[0]['summary_text']}\n{'-'*80}")


📰 Article 1 Summary:
This makes India the first country to reach this part of the Moon. The Indian Space Research Organisation (ISRO) celebrated this achievement,  highlighting that it demonstrates India’s growing capabilities in space exploration.
--------------------------------------------------------------------------------
📰 Article 2 Summary:
The agreement sets ambitious targets for reducing carbon emissions and aims to limit global warming to 1.5 degrees Celsius above pre-industrial levels. Nations have pledged to move away from fossil fuels, invest in renewable energy, and protect biodiversity.
--------------------------------------------------------------------------------
📰 Article 3 Summary:
Scientists have announced a major breakthrough in mRNA vaccine technology. A new vaccine has shown remarkable success in protecting against a wide range of flu strains in early human trials. The study, published in the New England Journal of Medicine, is being hailed as a milestone in pr

In [20]:
import torch

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluate on test set
metrics = trainer.evaluate(test_dataset)
print("📊 Test Set Evaluation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {round(v, 4)}")


📊 Test Set Evaluation Metrics:
eval_loss: 0.1136
eval_rouge1: 50.18
eval_rouge2: 37.73
eval_rougeL: 45.38
eval_runtime: 235.912
eval_samples_per_second: 6.68
eval_steps_per_second: 1.67
epoch: 5.0


In [21]:
model.save_pretrained("./bart-summarizer-ilsum")
tokenizer.save_pretrained("./bart-summarizer-ilsum")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 12, 'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./bart-summarizer-ilsum/tokenizer_config.json',
 './bart-summarizer-ilsum/special_tokens_map.json',
 './bart-summarizer-ilsum/vocab.json',
 './bart-summarizer-ilsum/merges.txt',
 './bart-summarizer-ilsum/added_tokens.json',
 './bart-summarizer-ilsum/tokenizer.json')